In [2]:
pip install requests
pip install bs4
pip install html5lib

Note: you may need to restart the kernel to use updated packages.


In [60]:
pip install lxml

In [1]:
import numpy as np
import pandas as pd
import json
import lxml.html as lh
import requests
import urllib.request
import bs4 as bs
from sklearn.cluster import KMeans


In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [3]:
# html code from wikipedia
def scrape_table(name, cols):
    page = urllib.request.urlopen(url).read()
    soup= bs.BeautifulSoup(page,'lxml')
    table = soup.find('table', class_=name)
    header = [head.findAll(text=True)[0].strip() for head in table.find_all('th')]
    data = [[td.findAll(text=True)[0].strip() for td in tr.find_all('td')] for tr in table.find_all('tr')]
    data =[row for row in data if len(row) == cols]
    raw_df = pd.DataFrame(data,columns=header)
    return raw_df

In [4]:
def scrape_table_lxml(xPATH, cols):
    page= requests.get(url)
    doc =lh.fromstring(page.content)
    table_content = doc.xpath(xPath)
    for table in table_content: 
        headers=[th.text_content().strip() for th in table.xpath('//th')]
        headers= headers[0:3]
        data = [[td.text_content().strip() for td in tr.xpath('td')]
               for tr in table.xpath('//tbody/tr')]
        data = [row for row in data if len(row) == cols]
        raw_df = pd.DataFrame(data, columns = headers)
        return raw_df

In [15]:
raw_TorontoPostalCodes = scrape_table('wikitable',3)
raw_TorontoPostalCodes.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Postal Code    180 non-null    object
 1   Borough        180 non-null    object
 2   Neighbourhood  180 non-null    object
dtypes: object(3)
memory usage: 4.3+ KB


In [16]:
#Data cleaning
TorontoPostalCodes=raw_TorontoPostalCodes[~raw_TorontoPostalCodes['Borough'].isin(['Not assigned'])]

In [26]:
#Sort and reset Index 
TorontoPostalCodes = TorontoPostalCodes.sort_values(by=['Postal Code','Borough','Neighbourhood'], ascending=[1,1,1]).reset_index(drop=True)

In [28]:
TorontoPostalCodes = TorontoPostalCodes.groupby(['Postal Code','Borough'])['Neighbourhood'].apply(','.join).reset_index()

In [30]:
TorontoPostalCodes

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


In [31]:
TorontoPostalCodes.to_csv('Toronto.TASK_1_df.csv',index = False)

In [33]:
task1_data_csv = 'Toronto.TASK_1_df.csv'
TorontoPostalCodes = pd.read_csv(task1_data_csv).set_index("Postal Code")
TorontoPostalCodes.rename_axis("Postal Code", axis='index', inplace=True)
TorontoPostalCodes.head()

,Borough,Neighbourhood
Postal Code,,
M1B,Scarborough,"Malvern, Rouge"
M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
M1E,Scarborough,"Guildwood, Morningside, West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


In [36]:
toronto_geocsv ='https://cocl.us/Geospatial_data'
geocsv_data = pd.read_csv(toronto_geocsv).set_index("Postal Code")
geocsv_data.head()

,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


In [38]:
# Joining two data frames with mapped postcodes

toronto_neighborhoods = TorontoPostalCodes.join(geocsv_data)
toronto_neighborhoods.head()

,Borough,Neighbourhood,Latitude,Longitude
Postal Code,,,,
M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
M1G,Scarborough,Woburn,43.770992,-79.216917
M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [39]:
toronto_neighborhoods.to_csv('Toronto.TASK_2_df.csv', index = False)